In [1]:
import pandas as pd
import numpy as np

load the dataset

In [15]:
df = pd.read_csv('NetflixDataset.csv')

check for missing values

In [16]:
missing = df.isnull().sum().sort_values(ascending = False)
missing_percentage = (missing/len(df))*100

display both counts and percentage

In [17]:
missing_data = pd.DataFrame({'missing values': missing, "percentage": missing_percentage})
print(missing_data)

              missing values  percentage
Director                2388   30.658621
Cast                     718    9.218128
Country                  507    6.509180
Release_Date              10    0.128386
Rating                     7    0.089870
Show_Id                    0    0.000000
Category                   0    0.000000
Title                      0    0.000000
Duration                   0    0.000000
Type                       0    0.000000
Description                0    0.000000


fill missing text values with placeholders

In [22]:
df["Director"].fillna("Unknown", inplace=True) #Too many missing (30%), better to retain data with a placeholder.
df["Cast"].fillna("Unkown", inplace=True) #Nearly 10% missing. Better than dropping rows.
df["Country"].fillna("Unknown", inplace=True) #6.5% missing. Still worth keeping.

Convert Release_Date to datetime format

In [26]:
df["Release_Date"] = pd.to_datetime(df["Release_Date"], errors="coerce")

Drop rows where Release_Date is still NaN

In [28]:
df.dropna(subset=["Release_Date"], inplace=True) #Only 10 rows affected, safe to drop.

Fill Rating with default

In [31]:
df["Rating"].fillna("Not Rated", inplace=True) #Very few missing. Safe to impute.

Check again for missing values

In [33]:
print("\nMissing values after cleaning:\n")
print(df.isnull().sum())


Missing values after cleaning:

Show_Id         0
Category        0
Title           0
Director        0
Cast            0
Country         0
Release_Date    0
Rating          0
Duration        0
Type            0
Description     0
dtype: int64


*Clean and Standardize Duration Column*

The Duration column usually contains values like:

"90 min"

"1 Season"

"2 Seasons"

But:

Movies use minutes (e.g., "90 min")

TV Shows use seasons (e.g., "1 Season")

We want to:

1.Split this column into two columns:

duration_int → numeric value (90, 1, 2)

duration_type → unit of duration (min, Season, Seasons)

2.Convert this into a cleaner, analyzable form.



In [37]:
#Split the 'Duration' into number and type
df["Duration_int"] = df["Duration"].str.extract("(\\d+)").astype(int)
df["Duration_type"] = df["Duration"].str.extract("([a-zA-Z]+)")

Standardize type (e.g., convert 'Seasons' to 'Season')

In [38]:
df["Duration_type"] = df["Duration_type"].replace({"Seasons":"Season", "Mins":"Min"})

Preview the result

In [39]:
df[["Duration", "Duration_int", "Duration_type"]].head()

,Duration,Duration_int,Duration_type
0,4 Seasons,4,Season
1,93 min,93,min
2,78 min,78,min
3,80 min,80,min
4,123 min,123,min


*Clean the Country Column*
    
Why clean it?

Some rows have multiple countries like "United States, India"

Some rows might have missing values (already handled them earlier, but let’s be sure)

Goal:
Fill missing countries with "Unknown" if any are still left.

Extract the first country only (assume it represents the main production country).

Create a new column like main_country.



In [40]:
#Fill missing countries if any (already cleaned, but safe)
df['Country'] = df['Country'].fillna("Unknown")

In [41]:
#Extract only the first country listed
df['main_country'] = df['Country'].apply(lambda x: x.split(',')[0].strip())

In [42]:
# Preview the result
df[['Country', 'main_country']].head()

,Country,main_country
0,Brazil,Brazil
1,Mexico,Mexico
2,Singapore,Singapore
3,United States,United States
4,United States,United States


*Clean and Standardize the Category or Type Column*
    
This column (Type) tells us whether the content is a Movie or a TV Show.
This is already quite clean — but let's make sure of a few things:

Goal:
Check unique values in the Type column.

Standardize them (e.g., lowercase or title case).

Encode them if you want to use them in modeling later.

In [44]:
#Check unique values
print("Unique values in Type column:", df['Type'].unique())

Unique values in Type column: ['International TV Shows, TV Dramas, TV Sci-Fi & Fantasy'
 'Dramas, International Movies' 'Horror Movies, International Movies'
 'Action & Adventure, Independent Movies, Sci-Fi & Fantasy' 'Dramas'
 'International TV Shows, TV Dramas, TV Mysteries'
 'Horror Movies, International Movies, Thrillers' 'Dramas, Thrillers'
 'Crime TV Shows, International TV Shows, TV Dramas'
 'Crime TV Shows, Docuseries, International TV Shows'
 'Documentaries, International Movies, Sports Movies'
 'Independent Movies, Sci-Fi & Fantasy, Thrillers'
 'Dramas, International Movies, Thrillers'
 'International TV Shows, TV Dramas'
 'Comedies, Dramas, Independent Movies' 'Sports Movies'
 'Dramas, Independent Movies, International Movies'
 'Action & Adventure, Dramas, International Movies'
 'Anime Series, International TV Shows' 'Documentaries' 'Reality TV'
 'Documentaries, International Movies' 'TV Comedies'
 'Dramas, International Movies, Romantic Movies' 'Comedies'
 'Comedies, Romant

In [52]:
#Standardize (title case for consistency)
df['Type'] = df['Type'].str.title()

In [54]:
# Preview the result
df[['Type']].head()

,Type
0,"International Tv Shows, Tv Dramas, Tv Sci-Fi &..."
1,"Dramas, International Movies"
2,"Horror Movies, International Movies"
3,"Action & Adventure, Independent Movies, Sci-Fi..."
4,Dramas


Final Dataset Preview

In [57]:
df.head()
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
Index: 7691 entries, 0 to 7788
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Show_Id        7691 non-null   object        
 1   Category       7691 non-null   object        
 2   Title          7691 non-null   object        
 3   Director       7691 non-null   object        
 4   Cast           7691 non-null   object        
 5   Country        7691 non-null   object        
 6   Release_Date   7691 non-null   datetime64[ns]
 7   Rating         7691 non-null   object        
 8   Duration       7691 non-null   object        
 9   Type           7691 non-null   object        
 10  Description    7691 non-null   object        
 11  Duration_int   7691 non-null   int64         
 12  Duration_type  7691 non-null   object        
 13  main_country   7691 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(12)
memory usage: 901.3+ KB


,Show_Id,Category,Title,Director,Cast,Country,Release_Date,Rating,Duration,Type,Description,Duration_int,Duration_type,main_country
count,7691,7691,7691,7691,7691,7691,7691,7691,7691,7691,7691,7691.000000,7691,7691
unique,7689,2,7689,4047,6745,678,NaN,15,216,487,7671,NaN,2,82
top,s684,Movie,Backfire,Unknown,Unkown,United States,NaN,TV-MA,1 Season,Documentaries,Multiple women report their husbands as missin...,NaN,min,United States
freq,2,5379,2,2295,710,2516,NaN,2841,1606,334,3,NaN,5379,2842
mean,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-07 22:53:43.195943424,NaN,NaN,NaN,NaN,69.962294,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,2008-01-01 00:00:00,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-07 00:00:00,NaN,NaN,NaN,NaN,2.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-15 00:00:00,NaN,NaN,NaN,NaN,88.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-25 12:00:00,NaN,NaN,NaN,NaN,106.000000,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-16 00:00:00,NaN,NaN,NaN,NaN,312.000000,NaN,NaN


In [58]:
df.to_csv("netflix_cleaned.csv", index=False)